In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from pdpbox.pdp import pdp_interact, pdp_interact_plot, pdp_isolate, pdp_plot
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.impute import SimpleImputer
import sklearn.metrics as metrics
from xgboost import XGBClassifier, plot_importance
import uuid 
plt.rcParams['figure.dpi'] = 72


In [2]:
a = uuid.uuid1()

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
pcs = pd.read_csv('data/pcs.csv')

In [5]:
pcs.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:da93549663709303e69475436aef4317807...
1,size 261273026


In [6]:
pcs.drop("survey_year", axis=1, inplace=True)

KeyError: "['survey_year'] not found in axis"

In [ ]:
pcs.shape

Lets check out how many people on this sruvey has mental illness

In [ ]:
pcs['mental_illness'].value_counts(normalize=True)

In [ ]:
pcs['mental_illness'].hist()

In [ ]:
sns.countplot(x="living_situation", palette="ch:.25", data=pcs, )

The overwheming majority huh......

In [ ]:
pcs['age_group'].value_counts(normalize=True)

Children account for approximately 18 percent of mental illness patients

In [ ]:
fem = pcs[pcs['sex']=="FEMALE"].copy()
mle = pcs[pcs['sex']=="MALE"].copy()
pcs['sex'].value_counts(normalize=True)

In [ ]:
mle['mental_illness'].value_counts(normalize=True)

In [ ]:
fem['mental_illness'].value_counts(normalize=True)

In [ ]:
ssi_no = pcs[pcs['ssi_cash_assistance']=="NO"]
ssi_yes = pcs[pcs['ssi_cash_assistance']=="YES"]
pcs['ssi_cash_assistance'].value_counts(normalize=True)

In [ ]:
ssi_no['mental_illness'].value_counts(normalize=True)

In [ ]:
ssi_yes['mental_illness'].value_counts(normalize=True)

In [ ]:
blk = pcs[pcs['race']=="black"].copy()
white = pcs[pcs['race']=="white"]
pcs['race'].value_counts(normalize=True)


In [ ]:
blk = pcs[pcs['race']=="BLACK ONLY"]
blk['mental_illness'].value_counts(normalize=True)

In [ ]:
oth = pcs[pcs['race']=="OTHER"]
oth['mental_illness'].value_counts(normalize=True)

In [ ]:
mr = pcs[pcs['race']=="MULTI-RACIAL"]
mr['mental_illness'].value_counts(normalize=True)

In [ ]:
white = pcs[pcs['race']=="WHITE ONLY"]
white['mental_illness'].value_counts(normalize=True)

In [ ]:
his = pcs[(pcs['hispanic_ethnicity']=="YES, HISPANIC/LATINO") | (pcs['hispanic_ethnicity']=='YES')]

In [ ]:
his['mental_illness'].value_counts(normalize=True)

In [ ]:
pcs['hispanic_ethnicity'].value_counts(normalize=True)

In [ ]:
pcs['transgender'].value_counts(normalize=True)


In [ ]:
trn['mental_illness'].value_counts(normalize=True)

In [ ]:
trn = pcs[pcs['transgender']=="YES, TRANSGENDER"]

In [ ]:
pcs['program_category'].value_counts(normalize=True)


In [ ]:
emr = pcs[pcs['program_category']=='EMERGENCY']
dis_emr = pcs[pcs['program_category']!='EMERGENCY']

In [ ]:
dis_emr

In [ ]:
emr['mental_illness'].value_counts(normalize=True)

In [ ]:
dis_emr['mental_illness'].value_counts(normalize=True)

In [ ]:
pcs['serious_mental_illness'].value_counts(normalize=True)

Strangely enough, 84.3 percent of respondants have a serious mental illness, along with 94.9 percent who havve a mental illness. * note this is a data leakage issue in future modeling 

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
sns.countplot(x="living_situation", palette="ch:.25", data=pcs, )

In [ ]:
pcs['living_situation'].value_counts(normalize=True)

Most live in a private residence, but the second largest feature is other. With little info on that, I become curious as to the true state of 20% of the respondants

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
sns.countplot(y="household_composition", palette="ch:1.3,-.2,dark=.3", data=pcs, )
plt.xlabel("Count")
plt.ylabel("Living Status")

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
plt.xticks(rotation='vertical')
sns.countplot(x="employment_status", palette="ch:2.5,-.3,dark=.3", data=pcs)
plt.xlabel("Count")
# plt.ylabel("Employment Status")

In [ ]:
# pcs['three_digit_residence_zip_code'].value_counts(normalize=True)


In [ ]:

f, axes = plt.subplots( sharey=True, figsize=(11, 4))
plt.xticks(rotation='vertical')

sns.countplot(x="three_digit_residence_zip_code", palette="ch:3.5,-.3,dark=.3", data=pcs, )
plt.xlabel("Three Digit Zip")
plt.ylabel("Count")

In [ ]:
 l = pcs['three_digit_residence_zip_code'].value_counts(normalize=True)

In [ ]:
l.head(2).sum()

Almost 25% of the group are accounted for with two zipcodes. With a wikipedia search, we see that comprises Brooklyn and The Bronx.

In [ ]:
pcs['id'] = [ uuid.uuid1()  for k in pcs.index]

In [ ]:
target = 'serious_mental_illness'
X = pd.DataFrame(pcs.drop(columns=[target,'mental_illness', 'principal_diagnosis_class', 'additional_diagnosis_class' ]))
y = pd.DataFrame(pcs[target])

In [ ]:
X.shape

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, 
    y, 
    test_size=0.001,
    train_size=0.10, 
    stratify=y, 
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, 
    y_trainval, 
    test_size=0.05,
    train_size=0.10, 
    stratify=y_trainval, 
    random_state=42
)

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

In [ ]:
5464 + 2733 + 547

In [ ]:
pcs.shape

In [ ]:
8744/546492

In [ ]:

0.016000234221177988 * 100

In [ ]:
train_id = X_train['id']
val_id = X_val['id']
test_id = X_test['id']

X_train = X_train.drop('id', axis=1)
X_val= X_val.drop('id', axis=1)
X_test= X_test.drop('id', axis=1)

In [ ]:
x_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

y_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

In [ ]:
cols = X_train.columns
len(cols)

In [ ]:
def prepare_inputs(X_train, X_val, X_test):
    X_train_enc = pd.DataFrame(x_processor.fit_transform(X_train), columns=cols)
    X_val_enc = pd.DataFrame(x_processor.transform(X_val), columns=cols)
    X_test_enc = pd.DataFrame(x_processor.transform(X_test), columns=cols)
    return X_train_enc, X_val_enc, X_test_enc

def prepare_targets(y_train, y_val, y_test):
    y_train_enc = pd.DataFrame(y_processor.fit_transform(y_train), columns=['serious_mental_illness'])
    y_val_enc = pd.DataFrame(y_processor.transform(y_val), columns=['serious_mental_illness'])
    y_test_enc = pd.DataFrame(y_processor.transform(y_test), columns=['serious_mental_illness'])
    return y_train_enc, y_val_enc, y_test_enc

In [ ]:
X_train_processed, X_val_processed, X_test = prepare_inputs(X_train, X_val, X_test)

In [ ]:
X_train_processed.head()

In [ ]:
y_train_processed, y_val_processed, y_test_processed = prepare_targets(y_train, y_val, y_test)

In [ ]:
y_train_processed.head()

In [ ]:
num_of_classes = len(y.serious_mental_illness.unique())
num_of_classes

In [ ]:
eval_set = [(X_train_processed, y_train_processed), (X_val_processed, y_val_processed)]

In [ ]:
xgbcl = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,  num_class=3,
         missing=None, n_jobs=-1, objective='multi:softprob', random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto')

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

param_grid = { 
    'colsample_bytree':[.75,1],
    'learning_rate':[0.01,0.05,0.1,0.3,0.5],
    'max_depth':[1,2,3,5],
    'subsample':[.75,1],
    'n_estimators': list(range(50, 400, 50))
}

In [ ]:
grid_search = GridSearchCV(estimator=xgbcl, param_grid=param_grid, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_processed, y_train_processed, eval_metric='mlogloss')

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
xgbcl = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='multi:softprob', num_class=3,random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto',
         colsample_bytree = grid_result.best_params_['colsample_bytree'], 
         learning_rate = grid_result.best_params_['learning_rate'], 
         max_depth = grid_result.best_params_['max_depth'], 
         subsample = grid_result.best_params_['subsample'], 
         n_estimators = grid_result.best_params_['n_estimators'], eval_metric="auc")

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(estimator=xgbcl, X=X_train_processed, y=y_train_processed, cv=kfold, n_jobs=-1)

In [ ]:
from sklearn import metrics
import joblib 
xgbcl.fit(X_train_processed, y_train_processed)


#Generate predictions against our training and test data
pred_train = xgbcl.predict(X_train_processed)
proba_train = xgbcl.predict_proba(X_train_processed)
pred_val = xgbcl.predict(X_val_processed)
proba_val = xgbcl.predict_proba(X_val_processed)
pred_test = xgbcl.predict(X_test)
proba_test = xgbcl.predict_proba(X_test)

In [ ]:
from joblib import dump, load
dump(xgbcl, 'xgbmodeal_v1_4.joblib')

In [ ]:
# clf = load('filename.joblib') 

In [ ]:
# pred_train = clf.predict(X_train_processed)

In [ ]:
print ('\nTrain loss:', metrics.log_loss(y_train_processed, proba_train))
print ('\nValidation loss:', metrics.log_loss(y_val_processed, proba_val))
print ('\nTest loss:', metrics.log_loss(y_test_processed, proba_test))

In [ ]:
print ('\nTrain Accuracy:', metrics.accuracy_score(y_train_processed, pred_train))
print ('\nValidation Accuracy:', metrics.accuracy_score(y_val_processed, pred_val))
print ('\nTest Accuracy:', metrics.accuracy_score(y_test_processed, pred_test))

In [ ]:
print("Classification report (Test): \n")
print(metrics.classification_report(y_test_processed, pred_test))

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 11))
plot_importance(xgbcl, ax=axes,importance_type='gain', max_num_features=10)

In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot

In [ ]:
pcs.ssi_cash_assistance.value_counts(normalize=True)

In [ ]:
pcs.ssdi_cash_assistance.value_counts(normalize=True)

In [ ]:
pcs.program_category.value_counts(normalize=True)


In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot

feature='ssi_cash_assistance'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, 
                           model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature);

In [ ]:

feature='autism_spectrum'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, 
                           model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:
pcs.received_smoking_counseling.value_counts(normalize=True)

In [ ]:

feature='received_smoking_counseling'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, 
                           model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:

feature='no_insurance'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, 
                           model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:

feature='medicaid_insurance'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, 
                           model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:
pcs.program_category.value_counts(normalize=True)

In [ ]:

feature='program_category'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:

feature='autism_spectrum'
pdp_isolated = pdp_isolate(model=xgbcl, dataset=X_train_processed, model_features=X_train_processed.columns, feature=feature)
pdp_plot(pdp_isolated, feature)

In [ ]:
res = pcs[pcs['program_category']=='RESIDENTIAL']

In [ ]:
res.serious_mental_illness.value_counts(normalize=True)

In [ ]:
non_res = pcs[pcs['program_category']!='RESIDENTIAL']

In [ ]:
non_res.serious_mental_illness.value_counts(normalize=True)


In [ ]:
X_train_processed.program_category.value_counts(normalize=True)

In [ ]:
X_train.program_category.value_counts(normalize=True)

In [ ]:
y_train_processed.serious_mental_illness.value_counts(normalize=True)

In [ ]:
y.serious_mental_illness.value_counts(normalize=True)

In [ ]:
X_train_processed.ssi_cash_assistance.value_counts(normalize=True)

In [ ]:
X_train.ssi_cash_assistance.value_counts(normalize=True)

In [ ]:
features = ['program_category', 'ssi_cash_assistance']

interaction = pdp_interact(
    model=xgbcl,
    dataset=X_train_processed,
    model_features=X_train_processed.columns,
    features=features
)

In [ ]:
# looking at how top features intereact
pdp = interaction.pdp.pivot_table(
    values='preds', 
    columns=features[0], # First feature on x axis
    index=features[1]    # Next feature on y axis
)[::-1]  # Reverse the index order so y axis is ascending

pdp = pdp.rename(columns=dict(zip(category_codes, category_names)))
plt.figure(figsize=(10,8))
sns.heatmap(pdp, annot=True, fmt='.2f', cmap='viridis')
plt.title('Partial Dependence of Serious Mental Illness, on program & ssi assistance')
pdp_interact_plot(interaction, plot_type='grid',feature_names=features)

In [ ]:
X_train.autism_spectrum.value_counts(normalize=True)

In [ ]:
X_train_processed.autism_spectrum.value_counts(normalize=True)

In [ ]:
features = ['program_category', 'ssi_cash_assistance']

interaction = pdp_interact(
    model=xgbcl,
    dataset=X_train_processed,
    model_features=X_train_processed.columns,
    features=features
)
pdp_interact_plot(interaction, plot_type='grid',feature_names=features)


In [ ]:
import shap

In [ ]:


#Attempt to use SHAP on multi-class
X_rand = X.sample(1, random_state=42)
idx = X_rand.index.values[0]

#This line will not work for a multi-class model, so we comment out
#explainer = shap.TreeExplainer(mcl, model_output='probability', feature_dependence='independent', data=X)

explainer = shap.TreeExplainer(xgbcl)
shap_values = explainer.shap_values(X.iloc[idx])
shap.initjs()
for which_class in range(0,3):
    display(shap.force_plot(explainer.expected_value[which_class], shap_values[which_class], X_rand))
    
#Display all features and SHAP values
df1=pd.DataFrame(data=shap_values[0], columns=X.columns, index=[0])
df2=pd.DataFrame(data=shap_values[1], columns=X.columns, index=[1])
df3=pd.DataFrame(data=shap_values[2], columns=X.columns, index=[2])
df=pd.concat([df1,df2,df3])
display(df.transpose())

